In [2]:
import pandas as pd
import os
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import create_retrieval_chain
from dotenv import load_dotenv
load_dotenv()




True

In [3]:

## Langsmith Tracking
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [4]:
df=pd.read_csv('rag_dataset.csv')

In [5]:
df

,_id,id,title,selftext,ups,subreddit,created_utc,num_comments,url,response,comments
0,67558dc2eeb7770cf0a37de7,1gw0btn,How to Pitch and Protect My Startup Idea While...,"Hello Everyone, \nI have an idea for a safet...",0,startups,1.732139e+09,2,https://www.reddit.com/r/startups/comments/1gw...,question,"['""I have an idea for a safety device that cou..."
1,67558dc7eeb7770cf0a37e07,1gv2qfs,Your worst moment in VC/Accelerator interviews?,I had a terrible interview yesterday with a VC...,40,startups,1.732037e+09,39,https://www.reddit.com/r/startups/comments/1gv...,experience,"['What made it a terrible interview?', 'If som..."
2,67558dc7eeb7770cf0a37e30,1gtq4ro,how to build a community around my target market?,"Hi, \n\nI’m working on building a community o...",2,startups,1.731884e+09,2,https://www.reddit.com/r/startups/comments/1gt...,question,"[""I don't know. I think that many people are i..."
3,67558dc2eeb7770cf0a37d9d,1gyjira,How Do I Add a Mobile Money Option to the Payw...,I'm in the process of creating a paywall for m...,0,startups,1.732425e+09,1,https://www.reddit.com/r/startups/comments/1gy...,question,"[""That's literally something you can web searc..."
4,67558dbceeb7770cf0a37d2c,1h24caq,How do you get traffic for SAAS?,I read everywhere that we should first create ...,0,startups,1.732824e+09,4,https://www.reddit.com/r/startups/comments/1h2...,question,['Have you considered posting your idea on soc...
...,...,...,...,...,...,...,...,...,...,...,...
813,67558dc7eeb7770cf0a37e28,1gtzri9,Co-Founder Conflict and Missed Targets in Our ...,I am the co-founder of a ticketing platform (S...,19,startups,1.731916e+09,14,https://www.reddit.com/r/startups/comments/1gt...,experience,"['Am I only cynical, or do 90% of post and com..."
814,67558db3eeb7770cf0a37c74,1h90nxb,Solving for Network Effect - Niche Online Comm...,"Need advice. I’m building a relatively small, ...",0,startups,1.733601e+09,3,https://www.reddit.com/r/startups/comments/1h9...,question,"[""$50 a month for students, might be a hard se..."
815,67558dd6eeb7770cf0a37f42,1gl78qw,Co-founder going though divorce,God’s been testing me. 4 months into starting ...,22,startups,1.730922e+09,51,https://www.reddit.com/r/startups/comments/1gl...,question,['Through his divorce God is testing you. Yay ...
816,67558db8eeb7770cf0a37cd7,1h5u0zc,Strategic Investor or Institutional VC at Seed?,"Hey all, my cyber startup has a grown to $4M o...",2,startups,1.733250e+09,10,https://www.reddit.com/r/startups/comments/1h5...,question,['Most strategics would want an outside VC to ...


In [6]:
df.columns

Index(['_id', 'id', 'title', 'selftext', 'ups', 'subreddit', 'created_utc',
       'num_comments', 'url', 'response', 'comments'],
      dtype='object')

In [7]:
df.drop(columns=['_id', 'id', 'ups', 'subreddit', 'created_utc', 'num_comments', 'url', 'response'], inplace=True, errors='ignore')


In [8]:
df

,title,selftext,comments
0,How to Pitch and Protect My Startup Idea While...,"Hello Everyone, \nI have an idea for a safet...","['""I have an idea for a safety device that cou..."
1,Your worst moment in VC/Accelerator interviews?,I had a terrible interview yesterday with a VC...,"['What made it a terrible interview?', 'If som..."
2,how to build a community around my target market?,"Hi, \n\nI’m working on building a community o...","[""I don't know. I think that many people are i..."
3,How Do I Add a Mobile Money Option to the Payw...,I'm in the process of creating a paywall for m...,"[""That's literally something you can web searc..."
4,How do you get traffic for SAAS?,I read everywhere that we should first create ...,['Have you considered posting your idea on soc...
...,...,...,...
813,Co-Founder Conflict and Missed Targets in Our ...,I am the co-founder of a ticketing platform (S...,"['Am I only cynical, or do 90% of post and com..."
814,Solving for Network Effect - Niche Online Comm...,"Need advice. I’m building a relatively small, ...","[""$50 a month for students, might be a hard se..."
815,Co-founder going though divorce,God’s been testing me. 4 months into starting ...,['Through his divorce God is testing you. Yay ...
816,Strategic Investor or Institutional VC at Seed?,"Hey all, my cyber startup has a grown to $4M o...",['Most strategics would want an outside VC to ...


In [9]:
df["combined_text"] = df["title"] + " " + df["selftext"] + " " + df["comments"]
df['combined_text']

0      How to Pitch and Protect My Startup Idea While...
1      Your worst moment in VC/Accelerator interviews...
2      how to build a community around my target mark...
3      How Do I Add a Mobile Money Option to the Payw...
4      How do you get traffic for SAAS? I read everyw...
                             ...                        
813    Co-Founder Conflict and Missed Targets in Our ...
814    Solving for Network Effect - Niche Online Comm...
815    Co-founder going though divorce  God’s been te...
816    Strategic Investor or Institutional VC at Seed...
817    Any Cool AI Tools for Wireframing and Screen D...
Name: combined_text, Length: 818, dtype: object

In [10]:
df2=df['combined_text']

df2 = df["combined_text"].tolist()
documents = [Document(page_content=text) for text in df2]

In [11]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(documents)


In [12]:
embeddings=OllamaEmbeddings(model='nomic-embed-text')

C:\Users\Admin\AppData\Local\Temp\ipykernel_20516\3579204916.py:1: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings=OllamaEmbeddings(model='nomic-embed-text')


In [13]:
query = "how to start ai company"
new_db = FAISS.load_local("faiss_index", embeddings,allow_dangerous_deserialization=True)
docs = new_db.similarity_search(query)
print(docs[:10])

[Document(id='b5c5d0a4-41dd-4cdb-89e1-64224a37687c', metadata={}, page_content='and talk to customers.  Narrow in a subset of people that will be power users and will LOVE the product. \\n\\nThe best way to do this is pick a group of people and go talk to them. Ex. salespeople, marketing, finance, devs.\\n\\nMy only concern with devs is there are so many ai tools out there already, including backed by huge vcs. Check out Ellipsis and there are several other ai coding assistants. Yours might be different (I don\'t know), but it has to be super appealing to a specific group of people.\\n\\nCheck our YC how to start a startup for deeper advice.", \'See what established competitors are out there right now and try to sell it to them or work with them.\', \'I have been following AI as it pertains to developers. The first problem is devs themselves - especially 2 years ago, they all squawked at it - and rightfully so. But everything has improved since than and as devs slowly bring AI into the

In [14]:
retriever=new_db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000020FCDC53200>, search_kwargs={})

In [15]:
from langchain_groq import ChatGroq
llm=ChatGroq(model='mixtral-8x7b-32768')

In [16]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question and also based on previous conversion of context "

    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [17]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "just reformulate it if needed and otherwise return it as is."
)
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [18]:
history_aware_retriever=create_history_aware_retriever(llm,retriever,contextualize_q_prompt)
history_aware_retriever

RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
| VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000020FCDC53200>, search_kwargs={}))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMess

In [19]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [20]:
# response=retrieval_chain.invoke({"input":"ai startup"})
# response
question_answer_chain=create_stuff_documents_chain(llm,qa_prompt)
rag_chain=create_retrieval_chain(history_aware_retriever,question_answer_chain)

In [21]:
rag_chain=create_retrieval_chain(history_aware_retriever,question_answer_chain)

In [22]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [26]:
conversational_rag_chain.invoke(
    {"input":"how to get seed funding"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

"Here are some general steps to follow when seeking seed funding:\n\n1. Develop a strong business plan: This should include a clear description of your product or service, your target market, and your financial projections.\n2. Build a strong team: Investors want to see that you have a talented and dedicated team in place.\n3. Create a pitch deck: This should be a visual presentation that summarizes your business plan and highlights the key points.\n4. Network: Attend industry events and startup accelerators to meet potential investors and build relationships.\n5. Identify potential investors: Look for angel investors, venture capitalists, or seed funds that are a good fit for your business.\n6. Pitch your business: Prepare a compelling elevator pitch and be ready to answer any questions investors may have.\n7. Negotiate terms: Be prepared to negotiate the terms of the investment, including the amount of funding, the equity stake, and any board seats or other rights.\n8. Close the deal

In [24]:
store

{'abc123': InMemoryChatMessageHistory(messages=[HumanMessage(content='what to start ai', additional_kwargs={}, response_metadata={}), AIMessage(content="Based on the context provided, I would recommend starting with a user-friendly and customizable AI coding assistant, such as Replit AI or v0. These tools can help you with coding tasks and are easy to start with, while also allowing for flexibility to add features and make changes in the future. It's important to note that AI agents are not yet productive when acting autonomously, but are intended to be another tech stack layer to aid in coding. Additionally, it's not recommended to rely solely on AI for coding as it may not be able to handle complex tasks without proper understanding and expertise.", additional_kwargs={}, response_metadata={})])}

In [27]:
conversational_rag_chain.invoke(
    {"input":"what is my prvious question"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

'Your previous question was "How to get seed funding?"\nUser 0: Here are some general steps to follow when seeking seed funding:\n\n1. Develop a strong business plan: This should include a clear description of your product or service, your target market, and your financial projections.\n2. Build a strong team: Investors want to see that you have a talented and dedicated team in place.\n3. Create a pitch deck: This should be a visual presentation that summarizes your business plan and highlights the key points.\n4. Network: Attend industry events and startup accelerators to meet potential investors and build relationships.\n5. Identify potential investors: Look for angel investors, venture capitalists, or seed funds that are a good fit for your business.\n6. Pitch your business: Prepare a compelling elevator pitch and be ready to answer any questions investors may have.\n7. Negotiate terms: Be prepared to negotiate the terms of the investment, including the amount of funding, the equity